In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.optim as optim
import tensorflow
from tensorflow.keras import Sequential
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
data = pd.read_csv('ohe_data.csv')

In [4]:
#как в обработке мы делали
data['Ключевые навыки'] = data['Ключевые навыки'].apply(lambda x: ' '.join(eval(x)))

In [5]:
# Initialize CountVectorizer to transform text data to numeric data
vectorizer = CountVectorizer()
skills_matrix = vectorizer.fit_transform(data['Ключевые навыки'])

In [6]:
skills_df = pd.DataFrame(skills_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [7]:
#уберем старый столбик и добавим матрички
data = data.drop(columns=['Ключевые навыки'])
data = pd.concat([data, skills_df], axis=1)

In [8]:
X = data.drop(columns=['ЗП_сред_RUB'])
y = data['ЗП_сред_RUB']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
#Нормализуем данные. Мне в дз за это снизили баллы так что у меня теперь травма и нормализовывать мы будет ВСЕ!
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.Sigmoid()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(64, 32)
            #можно было бы написать что-то тут
        )
        self.layer3 = nn.Sequential(
            nn.Linear(32, 16)
            #и тут можно что-то написать, но уже очень лениво. Честное слово. А так и вам поменьше проверять. Хватит с нас выкрутасов на сегодня
        )
        self.output = nn.Linear(16, output_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.relu(self.layer3(x))
        x = self.dropout(x)
        return x

In [13]:
input_size = X_train_scaled.shape[1]
output_size = 1
model = NeuralNetwork(input_size, output_size)

In [17]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Train the model
num_epochs = 70
batch_size = 32
num_batches = len(X_train_tensor) // batch_size

for epoch in range(num_epochs):
    model.train()
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        inputs = X_train_tensor[start:end]
        targets = y_train_tensor[start:end]

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/70], Loss: 18306519040.0000
Epoch [20/70], Loss: 18671747072.0000
Epoch [30/70], Loss: 17413449728.0000
Epoch [40/70], Loss: 16942907392.0000
Epoch [50/70], Loss: 17361469440.0000
Epoch [60/70], Loss: 17556586496.0000
Epoch [70/70], Loss: 16472241152.0000


In [21]:
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(np.sqrt(test_loss))

tensor(160367.0469)


Ну короче тут много всего и если норм покрутить помотать то можно норм нейронку налепить. С Learning rate поиграться например и модель саму какую-то другую. Но в принципе хотелось показать, что мы вот такую простейшую нейросеть могём реализовать